In [1]:
import pandas as pd
import sqlite3

In [3]:
conn = sqlite3.connect('../data/checking-logs.sqlite')

## получите схему средства проверки таблиц

In [4]:
schema_query = "PRAGMA table_info(checker);"
schema = pd.io.sql.read_sql(schema_query, conn)
print(schema)

   cid       name       type  notnull dflt_value  pk
0    0      index    INTEGER        0       None   0
1    1     status       TEXT        0       None   0
2    2    success    INTEGER        0       None   0
3    3  timestamp  TIMESTAMP        0       None   0
4    4  numTrials    INTEGER        0       None   0
5    5    labname       TEXT        0       None   0
6    6        uid       TEXT        0       None   0


## получите только первые 10 строк средства проверки таблицы, чтобы проверить, как выглядит таблица

In [5]:
first_10_rows_query = "SELECT * FROM checker LIMIT 10;"
first_10_rows = pd.io.sql.read_sql(first_10_rows_query, conn)
print(first_10_rows)

   index    status  success                   timestamp  numTrials   labname  \
0      0  checking        0  2020-04-16 21:12:50.740474          5      None   
1      1     ready        0  2020-04-16 21:12:54.708365          5  code_rvw   
2      2  checking        0  2020-04-16 21:46:47.769088          7      None   
3      3     ready        0  2020-04-16 21:46:48.121217          7     lab02   
4      4  checking        0  2020-04-16 21:53:01.862637          6  code_rvw   
5      5     ready        0  2020-04-16 21:53:05.373389          6  code_rvw   
6      6  checking        0  2020-04-17 05:18:51.965864          1      None   
7      7     ready        0  2020-04-17 05:19:02.744528          1  project1   
8      8  checking        0  2020-04-17 05:22:35.249331          2  project1   
9      9     ready        1  2020-04-17 05:22:45.549397          2  project1   

       uid  
0  admin_1  
1  admin_1  
2  admin_1  
3  admin_1  
4  admin_1  
5  admin_1  
6     None  
7   user_4  
8 

## подсчитайте, сколько строк удовлетворяет следующим условиям, используя только один запрос с любым количеством подзапросов:
* подсчитайте строки из таблицы просмотров страниц, но только для пользователей из таблицы проверки с:
1. status = "готово", мы не хотим анализировать журналы, которые находятся в состоянии проверки
2. numTrials = 1, мы хотим проанализировать только первые фиксации, потому что только они могут сказать нам, когда студент начал работать в лаборатории
3. имена лабораторий должны быть из списка: ’laba04’, ’laba04s’, ’laba05’, ’laba06’, ’laba06s’, ’project1’. Только те, которые были активны во время эксперимента
* сохраняйте в dataframe проверки с помощью столбца cnt

In [7]:
count_query = """
SELECT COUNT(*) AS cnt
FROM pageviews
WHERE uid IN (
    SELECT uid
    FROM checker
    WHERE status = 'ready'
    AND numTrials = 1 
    AND labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
    )
"""

checkers = pd.io.sql.read_sql(count_query, conn)
print(checkers)

   cnt
0  985


In [8]:
conn.close()